In [51]:
# import os
# os.chdir("../")
# %pwd

In [52]:

# model_trainer:
#   root_dir: artifacts/model_trainer
#   data_path: artifacts/data_transformation/samsum_dataset
#   model_ckpt: google/pegasus-cnn_dailymail

from dataclasses import dataclass
from pathlib import Path

In [53]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int


In [54]:
from src.summarizer.constants import *
from src.summarizer.utils import *
from src.summarizer.exception import CustomException

In [55]:
from pathlib import Path
from src.summarizer.constants import *
from src.summarizer.utils import read_yaml, create_directories
from src.summarizer.exception import CustomException
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_PATH, params_filepath=PARAMS_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        return ModelTrainerConfig(
            root_dir=Path(config.root_dir),
            data_path=Path(config.data_path),
            model_ckpt=Path(config.model_ckpt),
            num_train_epochs=int(params.num_train_epochs),
            warmup_steps=int(params.warmup_steps),
            per_device_train_batch_size=int(params.per_device_train_batch_size),
            weight_decay=float(params.weight_decay),
            logging_steps=int(params.logging_steps),
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=int(params.eval_steps),
            save_steps=float(params.save_steps),
            gradient_accumulation_steps=int(params.gradient_accumulation_steps)
        )


In [56]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

In [57]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_from_disk
from src.summarizer.constants import * 
import os

# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM



class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        # Load model directly
        from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

        tokenizer = AutoTokenizer.from_pretrained("Falconsai/text_summarization")
        model = AutoModelForSeq2SeqLM.from_pretrained("Falconsai/text_summarization")
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
        
        # Load data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # Initialize TrainingArguments
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay,
            logging_dir=os.path.join(self.config.root_dir, 'logs'),
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps,
            save_steps=int(self.config.save_steps),  # Ensure this is an integer
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        )

        trainer = Trainer(
            model=model,
            args=trainer_args,
            tokenizer=tokenizer,
            data_collator=seq2seq_data_collator,
            train_dataset=dataset_samsum_pt["train"], 
            eval_dataset=dataset_samsum_pt["validation"]
        )
        
        trainer.train()

        # Save model and tokenizer
        model.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))


In [59]:
import sys
from src.summarizer.exception import CustomException

if __name__ == "__main__":
    try:
        config = ConfigurationManager()
        model_trainer_config = config.get_model_trainer_config()
        trainer = ModelTrainer(config=model_trainer_config)
        trainer.train()
    except Exception as e:
        raise CustomException(e, sys)


[2024-08-16 13:56:48,594: INFO: __init__: yaml file: config\config.yaml loaded successfully]
[2024-08-16 13:56:48,607: INFO: __init__: yaml file: params.yaml loaded successfully]
[2024-08-16 13:56:48,609: INFO: __init__: created directory at: artifacts]
[2024-08-16 13:56:48,615: INFO: __init__: created directory at: artifacts/model_trainer]
[2024-08-16 13:56:50,955: INFO: __init__: Error: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U` | File: C:\Users\riswa\AppData\Local\Temp\ipykernel_16272\808809606.py | Line: 9]
[2024-08-16 13:56:50,955: INFO: __init__: Traceback (most recent call last):
  File "C:\Users\riswa\AppData\Local\Temp\ipykernel_16272\808809606.py", line 9, in <module>
    trainer.train()
  File "C:\Users\riswa\AppData\Local\Temp\ipykernel_16272\3783665554.py", line 30, in train
    trainer_args = TrainingArguments(
  File "<string>", line 131, in __init__
  File "c:\Users\riswa\

CustomException: Error: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U` | File: C:\Users\riswa\AppData\Local\Temp\ipykernel_16272\808809606.py | Line: 9

In [61]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

from src.summarizer.constants import *
from src.summarizer.utils import read_yaml, create_directories
from src.summarizer.exception import CustomException

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_PATH, params_filepath=PARAMS_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        return ModelTrainerConfig(
            root_dir=Path(config.root_dir),
            data_path=Path(config.data_path),
            model_ckpt=Path(config.model_ckpt),
            num_train_epochs=int(params.num_train_epochs),
            warmup_steps=int(params.warmup_steps),
            per_device_train_batch_size=int(params.per_device_train_batch_size),
            weight_decay=float(params.weight_decay),
            logging_steps=int(params.logging_steps),
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=int(params.eval_steps),
            save_steps=float(params.save_steps),
            gradient_accumulation_steps=int(params.gradient_accumulation_steps)
        )

from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_from_disk
import torch
import os

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = torch.device("cpu")  # Explicitly set device to CPU

        # Load model directly
        tokenizer = AutoTokenizer.from_pretrained("Falconsai/text_summarization")
        model = AutoModelForSeq2SeqLM.from_pretrained("Falconsai/text_summarization").to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

        # Load data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # Initialize TrainingArguments
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay,
            logging_dir=os.path.join(self.config.root_dir, 'logs'),
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps,
            save_steps=int(self.config.save_steps),
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        )

        trainer = Trainer(
            model=model,
            args=trainer_args,
            tokenizer=tokenizer,
            data_collator=seq2seq_data_collator,
            train_dataset=dataset_samsum_pt["train"],
            eval_dataset=dataset_samsum_pt["validation"]
        )

        trainer.train()

        # Save model and tokenizer
        model.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

import sys
from src.summarizer.exception import CustomException

if __name__ == "__main__":
    try:
        config = ConfigurationManager()
        model_trainer_config = config.get_model_trainer_config()
        trainer = ModelTrainer(config=model_trainer_config)
        trainer.train()
    except Exception as e:
        raise CustomException(e, sys)


[2024-08-16 14:02:08,655: INFO: __init__: yaml file: config\config.yaml loaded successfully]
[2024-08-16 14:02:08,679: INFO: __init__: yaml file: params.yaml loaded successfully]
[2024-08-16 14:02:08,687: INFO: __init__: created directory at: artifacts]
[2024-08-16 14:02:08,693: INFO: __init__: created directory at: artifacts/model_trainer]


[2024-08-16 14:02:10,642: INFO: __init__: Error: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U` | File: C:\Users\riswa\AppData\Local\Temp\ipykernel_16272\156252809.py | Line: 109]
[2024-08-16 14:02:10,644: INFO: __init__: Traceback (most recent call last):
  File "C:\Users\riswa\AppData\Local\Temp\ipykernel_16272\156252809.py", line 109, in <module>
    trainer.train()
  File "C:\Users\riswa\AppData\Local\Temp\ipykernel_16272\156252809.py", line 71, in train
    trainer_args = TrainingArguments(
  File "<string>", line 131, in __init__
  File "c:\Users\riswa\anaconda3\envs\summarizer\lib\site-packages\transformers\training_args.py", line 1730, in __post_init__
    self.device
  File "c:\Users\riswa\anaconda3\envs\summarizer\lib\site-packages\transformers\training_args.py", line 2227, in device
    return self._setup_devices
  File "c:\Users\riswa\anaconda3\envs\summarizer\lib\site-packages\tr

CustomException: Error: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U` | File: C:\Users\riswa\AppData\Local\Temp\ipykernel_16272\156252809.py | Line: 109

In [73]:
from dataclasses import dataclass
from pathlib import Path
from transformers import TFAutoModelForSeq2SeqLM, TFTrainingArguments, AutoTokenizer, DataCollatorForSeq2Seq
from datasets import load_from_disk
import os

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_PATH, params_filepath=PARAMS_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        return ModelTrainerConfig(
            root_dir=Path(config.root_dir),
            data_path=Path(config.data_path),
            model_ckpt=Path(config.model_ckpt),
            num_train_epochs=int(params.num_train_epochs),
            warmup_steps=int(params.warmup_steps),
            per_device_train_batch_size=int(params.per_device_train_batch_size),
            weight_decay=float(params.weight_decay),
            logging_steps=int(params.logging_steps),
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=int(params.eval_steps),
            save_steps=float(params.save_steps),
            gradient_accumulation_steps=int(params.gradient_accumulation_steps)
        )
        
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_from_disk
import tensorflow as tf
import os

class ModelTrainer:
    def __init__(self, config):
        self.config = config

    def train(self):
        # Load model and tokenizer with TensorFlow
        tokenizer = AutoTokenizer.from_pretrained("Falconsai/text_summarization")
        model = TFAutoModelForSeq2SeqLM.from_pretrained("Falconsai/text_summarization")

        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # Prepare the datasets
        train_dataset = dataset_samsum_pt["train"].to_tf_dataset(
            columns=["input_ids", "attention_mask", "labels"],
            shuffle=True,
            batch_size=self.config.per_device_train_batch_size,
            collate_fn=lambda x: tokenizer.pad(x, return_tensors="tf"),
        )

        eval_dataset = dataset_samsum_pt["validation"].to_tf_dataset(
            columns=["input_ids", "attention_mask", "labels"],
            shuffle=False,
            batch_size=self.config.per_device_train_batch_size,
            collate_fn=lambda x: tokenizer.pad(x, return_tensors="tf"),
        )

        # Compile the model
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
            loss=model.compute_loss  # Model's loss function is directly used
        )

        # Train the model
        model.fit(
            train_dataset,
            validation_data=eval_dataset,
            epochs=self.config.num_train_epochs,
        )

        # Save model and tokenizer
        model.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))


import sys
from src.summarizer.exception import CustomException

if __name__ == "__main__":
    try:

        # trainer = ModelTrainer(config=model_trainer_config)
        # trainer.train()
        config = ConfigurationManager()
        model_trainer_config = config.get_model_trainer_config()
        model_trainer_config = ModelTrainer(config=model_trainer_config)
        model_trainer_config.train()
    except Exception as e:
        raise CustomException(e, sys)


[2024-08-16 15:43:47,344: INFO: __init__: yaml file: config\config.yaml loaded successfully]
[2024-08-16 15:43:47,360: INFO: __init__: yaml file: params.yaml loaded successfully]
[2024-08-16 15:43:47,410: INFO: __init__: created directory at: artifacts]
[2024-08-16 15:43:47,418: INFO: __init__: created directory at: artifacts/model_trainer]


[2024-08-16 15:43:49,586: INFO: __init__: Error: 
TFAutoModelForSeq2SeqLM requires the TensorFlow library but it was not found in your environment.
However, we were able to find a PyTorch installation. PyTorch classes do not begin
with "TF", but are otherwise identically named to our TF classes.
If you want to use PyTorch, please use those classes instead!

If you really do want to use TensorFlow, please follow the instructions on the
installation page https://www.tensorflow.org/install that match your environment.
 | File: C:\Users\riswa\AppData\Local\Temp\ipykernel_16272\1146439235.py | Line: 108]
[2024-08-16 15:43:49,591: INFO: __init__: Traceback (most recent call last):
  File "C:\Users\riswa\AppData\Local\Temp\ipykernel_16272\1146439235.py", line 108, in <module>
    model_trainer_config.train()
  File "C:\Users\riswa\AppData\Local\Temp\ipykernel_16272\1146439235.py", line 60, in train
    model = TFAutoModelForSeq2SeqLM.from_pretrained("Falconsai/text_summarization")
  File "c:\

CustomException: Error: 
TFAutoModelForSeq2SeqLM requires the TensorFlow library but it was not found in your environment.
However, we were able to find a PyTorch installation. PyTorch classes do not begin
with "TF", but are otherwise identically named to our TF classes.
If you want to use PyTorch, please use those classes instead!

If you really do want to use TensorFlow, please follow the instructions on the
installation page https://www.tensorflow.org/install that match your environment.
 | File: C:\Users\riswa\AppData\Local\Temp\ipykernel_16272\1146439235.py | Line: 108

In [69]:
import tensorflow as tf
print(tf.__version__)


2.13.0


In [65]:
import transformers
print(transformers.__version__)


4.44.0
